In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import pytest
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect

# Fixtures
from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import y
from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import exog, exog_predict

In [88]:
forecaster = ForecasterDirect(
                    regressor = LinearRegression(),
                    steps     = 5,
                    lags      = 5
                )
forecaster.fit(y=y)

recommended_n_boot = 5
forecaster.in_sample_residuals_by_bin_[1] = forecaster.in_sample_residuals_by_bin_[1][:recommended_n_boot]
for k, v in forecaster.in_sample_residuals_by_bin_.items():
    if k != 1:
        forecaster.in_sample_residuals_by_bin_[k] = v[:recommended_n_boot - 1]
    
results = forecaster.predict_bootstrapping(
    steps=5, n_boot=recommended_n_boot, use_in_sample_residuals=True, use_binned_residuals=True
)
results

8.0
[-0.46599771 -0.13167749 -0.2400648   0.38610882 -0.46599771]
4.0
[-0.06597795 -0.06247223 -0.30764313  0.11784057 -0.30764313]
2.0
[-0.0450625  -0.04970559 -0.27109241 -0.2054548  -0.27109241]
1.0
[ 0.01079722  0.31667074  0.02045466 -0.03134715 -0.18923835]
1.0
[ 0.01079722  0.31667074  0.02045466 -0.03134715 -0.18923835]


,pred_boot_0,pred_boot_1,pred_boot_2,pred_boot_3,pred_boot_4
50,0.087196,0.421516,0.313129,0.939302,0.087196
51,0.417073,0.420579,0.175408,0.600892,0.175408
52,0.404750,0.400107,0.178720,0.244358,0.178720
53,0.424213,0.730087,0.433870,0.382069,0.224177
54,0.430659,0.736532,0.440316,0.388514,0.230623


In [87]:
recommended_n_boot = 5
in_sample_residuals_by_bin_ = forecaster.in_sample_residuals_by_bin_.copy()
in_sample_residuals_by_bin_[1] = in_sample_residuals_by_bin_[1][:recommended_n_boot]
for k, v in in_sample_residuals_by_bin_.items():
    if k != 1:
        in_sample_residuals_by_bin_[k] = v[:recommended_n_boot - 1]

in_sample_residuals_by_bin_

{0: array([ 0.35398646, -0.01070607, -0.0121861 ,  0.06634067]),
 1: array([ 0.01079722,  0.31667074,  0.02045466, -0.03134715, -0.18923835]),
 2: array([-0.0450625 , -0.04970559, -0.27109241, -0.2054548 ]),
 3: array([ 0.01474554, -0.12607334, -0.40848077,  0.1781821 ]),
 4: array([-0.06597795, -0.06247223, -0.30764313,  0.11784057]),
 5: array([ 0.47165077, -0.01760355, -0.39344615,  0.47603728]),
 6: array([ 0.21672936,  0.00902857, -0.11833943,  0.20465353]),
 7: array([-0.01111603, -0.29143333, -0.10171245, -0.05190446]),
 8: array([-0.46599771, -0.13167749, -0.2400648 ,  0.38610882]),
 9: array([ 0.06586613,  0.27294283,  0.13207642, -0.05288698])}

In [85]:
results.to_numpy()

array([[0.0871959 , 0.42151611, 0.31312881, 0.93930242, 0.0871959 ],
       [0.41707327, 0.42057899, 0.17540809, 0.60089178, 0.17540809],
       [0.40475029, 0.4001072 , 0.17872038, 0.24435799, 0.17872038],
       [0.42421302, 0.73008654, 0.43387046, 0.38206866, 0.22417746],
       [0.43065853, 0.73653206, 0.44031597, 0.38851417, 0.23062297]])

In [11]:
forecaster.binner_intervals_

{0.0: (77.11104390221573, 95.19313158258132),
 1.0: (95.19313158258132, 107.10836783689707),
 2.0: (107.10836783689707, 122.98049619443195)}

In [4]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.exceptions import NotFittedError
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor

from skforecast.preprocessing import RollingFeatures
from skforecast.preprocessing import TimeSeriesDifferentiator
from skforecast.recursive import ForecasterRecursive
from scipy.stats import norm
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from skforecast.model_selection import TimeSeriesFold, backtesting_forecaster

# Fixtures
from skforecast.recursive.tests.tests_forecaster_recursive.fixtures_forecaster_recursive import y
from skforecast.recursive.tests.tests_forecaster_recursive.fixtures_forecaster_recursive import exog
from skforecast.recursive.tests.tests_forecaster_recursive.fixtures_forecaster_recursive import exog_predict
from skforecast.recursive.tests.tests_forecaster_recursive.fixtures_forecaster_recursive import data  # to test results when using differentiation


In [17]:
forecaster = ForecasterDirect(
        regressor=LinearRegression(), steps=5, lags=3, binner_kwargs={'n_bins': 10}
    )
forecaster = ForecasterRecursive(
        regressor=LinearRegression(), lags=3, binner_kwargs={'n_bins': 10}
    )
n_backtest = 12
y_train = y[:-n_backtest]
cv = TimeSeriesFold(
             steps              = 5,
             initial_train_size = len(y_train),
             refit              = True
        )
metric, backtest_predictions = backtesting_forecaster(
                                    forecaster              = forecaster,
                                    y                       = y,
                                    exog                    = exog,
                                    cv                      = cv,
                                    metric                  = 'mean_squared_error',
                                    interval                = 0.9,
                                    interval_method         = 'conformal',
                                    use_in_sample_residuals = True,
                                    use_binned_residuals    = True,
                                    random_state            = 123,
                                    verbose                 = False,
                                    show_progress           = False
                                )

In [18]:
# Increase number of decimals numpy
np.set_printoptions(precision=15)
metric.to_numpy()

array([[0.064250191230055]])

In [19]:
np.set_printoptions(precision=8)
backtest_predictions.to_numpy()

array([[0.59059622, 0.2974502 , 0.88374223],
       [0.47257504, 0.22356317, 0.72158691],
       [0.53024098, 0.14339956, 0.91708239],
       [0.46163343, 0.21262156, 0.7106453 ],
       [0.50035119, 0.35604735, 0.64465502],
       [0.41975558, 0.20182965, 0.63768151],
       [0.4256614 , 0.03773832, 0.81358449],
       [0.41176005, 0.19383413, 0.62968598],
       [0.52357817, 0.43392621, 0.61323014],
       [0.509974  , 0.42032204, 0.59962597],
       [0.65354628, 0.29763244, 1.00946013],
       [0.48210726, 0.17026625, 0.79394828]])